<h1>Preparation of the English Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from HanTa import HanoverTagger as ht

<p>Download der Stopwörter Listen</p>

In [ ]:
#nltk.download('stopwords')

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [2]:
df_manager = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_EN/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager = df_manager.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [3]:
len(df_manager)

17844

In [4]:
df_manager.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703142204341026818,349287204,2016-02-26 08:58:59+00:00,703142204341026818,en,0,0,1,0,Twitter Web Client,None,None,1/2 I see Wirecard is rebounding after CEO Mar...
1,695258744737824768,282981965,2016-02-04 14:52:56+00:00,695258744737824768,en,1,0,0,0,Twitter for iPhone,retweeted,695237701386244098,RT @UrgentSpeed: Disrupt the Banks! (Jeff Stew...
2,695255175334924288,428593285,2016-02-04 14:38:45+00:00,695255175334924288,en,0,0,0,0,IFTTT,None,None,Thank you for following us Markus Braun!
3,695237701386244098,95129352,2016-02-04 13:29:19+00:00,695237701386244098,en,1,0,1,0,Twitter Web Client,None,None,"Disrupt the Banks! (Jeff Stewart, Markus Braun..."
4,719940834028863489,3040068274,2016-04-12 17:30:45+00:00,719940834028863489,en,0,0,0,0,metal archives updates,None,None,Artist updated: Markus Braun https://t.co/udZM...


<h3>Löschen von unnötigen Informationen</h3>

<p>Hiermit werden alle Textbausteine gelöscht, die nicht benötigt werden. Personen bleiben in diesem Schritt noch enthalten.</p>

In [6]:
#just for test
for each in range(len(df_manager)):
    df_manager.loc[df_manager["text"] == 
                      df_manager.loc[each]["text"], "text"] = re.sub(r'http\S+', '',(re.sub('[#|$]', '', df_manager.loc[each]["text"])))

In [ ]:
"""for each in range(len(df_manager)):
    df_manager.loc[df_manager["text"] == 
                      df_manager.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_manager.loc[each]["text"]))))"""

<h1>Löschen von nicht weiter relevanten Tweets</h1>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [7]:
counter = 0
retweet_list = []
for each in range(len(df_manager)):
    if "retweeted" in df_manager.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

9473


In [8]:
df_manager = df_manager.drop(retweet_list)
df_manager = df_manager.reset_index(drop=True)

<h3>Löschen von doppelten Tweets</h3>
<p>Alle Tweets, die dopplet im Datensatz enthalten sind, anhand der tweet_id werden gelöscht. Somit werden auch die zuvor aufgespalteten Listen wieder gelöscht</p>

In [9]:
df_manager = df_manager.drop_duplicates(subset=["tweet_id"])
df_manager = df_manager.reset_index(drop=True)

In [10]:
len(df_manager)

8328

<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.<br>Die wörter in der Liste müssen klein geschrieben werden, da auch alle Buchstaben für die Prüfung klein geschrieben werden müssen, da sinst nicht alles erkannt wird.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>meiko</li>
    <li>calisthenics</li>
    <li>pulled</li>
    <li>boxing</li>
    <li>fom</li>
    <li>dlr</li>
    <li>eurocity</li>
    <li>m.o.s.s</li>
    <li>markus_braun_</li>
</ul>

<p>Überprüfung der englischen Tweets. Im ersten Schritt wurden die Retweets gelöscht, dann alle doppelten Tweets. Anschließend gab es eine Liste mit Begriffen, die definitiv etwas mit dem Thema zu tun hatten. Diese wurden aus dem Datensatz zur prüfung gelöscht. Darauf folgte eine Liste mit Begriffen, die nichts mit dem Thema zu tun haben und deswegen gelöscht werden. Abschließend wurden zu überprüfungszwecken Tweets mit doppelten Texten gelöscht und im Anschluss einzelne Tweet ID gelöscht.</p>

In [11]:
#List with single IDs, which are not relevant for the dataset
list_tweet_id = [719940834028863489, 771966867707682816, 919891703116566529, 730315192559673344, 1085012780053336064, 748146758232879104,
                 1275383293504978945, 1121414883512393728, 1288961015351316480, 1284465825915838464, 955660777960230917, 962669158923153409,
                 1016245557021507584, 1074388350734340097, 1115926018051792898, 1132148944031457280, 1130223973701246976, 1174984360669278209,
                 1220356450809188353, 1255176912202108929]

#Check and get tweet_ids for tweets with the same text as in the list_tweet_id.
for tweet_id in list_tweet_id:
    for each in df_manager.loc[df_manager["text"] == df_manager.loc[df_manager["tweet_id"] == tweet_id]["text"].values[0]]["tweet_id"].values:
        if each not in list_tweet_id:
            list_tweet_id.append(each)
            
#List of words which does not have anything to do with wirecard.
not_used_words_list = ["meiko", "calisthenics", "pulled pork", "boxing", " fom", "dlr", "eurocity", "m.o.s.s", "markus_braun_"]

#Iterate though the dataframe to get each tweet_id and save it in the list_tweet_id
for each in range(len(df_manager)):
    for word in not_used_words_list:
        if word in df_manager.iloc[each]["text"].lower():
            if df_manager.iloc[each]["tweet_id"] not in list_tweet_id:
                list_tweet_id.append(df_manager.iloc[each]["tweet_id"])


#Get further tweet_ids for every tweet_id that is stored in list_tweet_id
for tweet in list_tweet_id:
    for each in df_manager.loc[(
        df_manager["tweet_id"] == tweet) | (
        df_manager["conversation_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0]) | (
        df_manager["referenced_tweet_id"] == tweet) | (
        (df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager["referenced_tweet_id"] != "None")) | (
        df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0])]["tweet_id"].values:
        #If the value is not in the list, it will be added to the list_tweet_id
        if each not in list_tweet_id:
            list_tweet_id.append(each)

<p>Löschen der zuvor ermitteletn Tweets.</p>

In [12]:
#Iterate through list_tweet_id and remove every tweet, that has nothing to do with Wirecard.
for each in list_tweet_id:
    df_manager = df_manager.drop(df_manager.loc[df_manager["tweet_id"] == each].index[0])
#Set back the index. Otherwise it will not be possible to work with the index any loner. 
df_manager = df_manager.reset_index(drop=True)

In [13]:
for each in range(len(df_manager)):
    df_manager.loc[df_manager["text"] == 
                      df_manager.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_manager.loc[each]["text"]))))

<h2>Zur Prüfung</h2>

<p>So wurde der Datensatz immer weiter eingegrenzt auf die Begriffe, die relevant sind und unrelevant. Zudem wurde die Liste erstellt, mit einzelnen Tweets, die keinen Input liefern.</p>

In [ ]:
print(len(df_manager))

#remove retweets
retweet_list = []
for each in range(len(df_manager)):
    if "retweeted" in df_manager.iloc[each]["tweet_type"]:
        retweet_list.append(each)

df_manager = df_manager.drop(retweet_list)
df_manager = df_manager.reset_index(drop=True)

#remove duplicates
df_manager = df_manager.drop_duplicates(subset=["tweet_id"])
df_manager = df_manager.reset_index(drop=True)

#words that are relevant but needs to be removed for check
for each in df_manager.loc[df_manager.text.str.lower().str.contains("wirecard|_markusbraun|kpmg|wdi|bank|financial|@ft|dld|philippine|@fd|dan|turtleneck|ceo|criminal|tech|billion|bitcoin|wanted|bafin|ernst|money|pay|wcagy|arrest|handel|wire|marsalek|braun")].index:
    df_manager = df_manager.drop(each)
df_manager = df_manager.reset_index(drop=True)

#word that needs to be removed as they do not preserve any further value.
for each in df_manager.loc[df_manager.text.str.lower().str.contains("meiko|calisthenics|pulled|boxing|fom|dlr|eurocity|m.o.s.s|markus_braun_")].index:
    df_manager = df_manager.drop(each)
df_manager = df_manager.reset_index(drop=True)

#for checkup all tweets with the same text will be removed.
df_manager = df_manager.drop_duplicates(subset=["text"])
df_manager = df_manager.reset_index(drop=True)

#remove single tweet list
list_single_tweet_id = [719940834028863489, 771966867707682816, 919891703116566529, 730315192559673344, 1085012780053336064, 748146758232879104, 1275383293504978945, 1121414883512393728, 1288961015351316480, 1284465825915838464, 955660777960230917, 962669158923153409, 1016245557021507584, 1074388350734340097, 1115926018051792898, 1132148944031457280, 1130223973701246976, 1174984360669278209, 1220356450809188353, 1255176912202108929]
for each in list_single_tweet_id:
    if each in df_manager.loc[df_manager["tweet_id"] == each].values:
        df_manager = df_manager.drop(df_manager.loc[df_manager["tweet_id"] == each].index[0])
df_manager = df_manager.reset_index(drop=True)

print(len(df_manager))

In [ ]:
for each in range(len(df_manager)):
    print(df_manager.iloc[each]["tweet_id"], ": ", df_manager.iloc[each]["text"])

In [ ]:
for each in df_manager.loc[df_manager.text.str.lower().str.contains("ey")].index:
    print(df_manager.iloc[each]["tweet_id"], ": ", df_manager.iloc[each]["text"])

In [ ]:
for each in range(len(df_manager.sample(n=250, random_state=1))):
    print(df_manager.iloc[each]["tweet_id"], ": ", df_manager.iloc[each]["text"])

<p>Das vorherige Verfahren zum ermitteln weiterer Tweet_ids und deren zugehörigen Tweets. Das neue Verfahren, wie oben angewendet erzielt die selben Ergebnisse.</p>

In [ ]:
not_used_words_list = ["meiko", "calisthenics", "pulled pork", "boxing", " fom", "dlr", "eurocity", "m.o.s.s", "markus_braun_"]
tweet_id_without_context_list = []
for each in range(len(df_manager)):
    for word in not_used_words_list:
        if word in df_manager.iloc[each]["text"].lower():
            if df_manager.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_manager.iloc[each]["tweet_id"])

<p>Iteration durch die Liste mit zu löschenden Tweets, bis der Wert zu Beginn und zum Ende gleich ist und es keine
weiteren Tweets mehr gibt, die gefiltert werden müssen. </p>

In [ ]:
def check_for_further_tweets():
    start_len = len(tweet_id_without_context_list)
    for each in range(len(tweet_id_without_context_list)):
        df_tweets = df_manager.loc[(df_manager["conversation_id"] == tweet_id_without_context_list[each]) | 
               (df_manager["referenced_tweet_id"] == str(tweet_id_without_context_list[each]))]

        for each in range(len(df_tweets)):
            if df_tweets.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_tweets.iloc[each]["tweet_id"])
                
    if start_len != len(tweet_id_without_context_list):
        check_for_further_tweets()
check_for_further_tweets()

<p>Löschen der nicht kontextbezogenen Tweets und den Index neu setzen um später keine Probleme mit dem Index zu bekommen.</p>

In [ ]:
for each in tweet_id_without_context_list:
    df_manager = df_manager.drop(df_manager.loc[df_manager["tweet_id"] == each].index[0])
df_manager = df_manager.reset_index(drop=True)

<h1>Hinzufügen weiterer Twitter Datensätze</h1>
<p>Hier werden die weiteren Datensätze zusammengeführt.</p>

<h2>Datensatz mit Abfrage nach Wirecard</h2>

In [14]:
df_wirecard = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Wirecard_EN/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wirecard = df_wirecard.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [15]:
for each in range(len(df_wirecard)):
    df_wirecard.loc[df_wirecard["text"] == 
                      df_wirecard.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_wirecard.loc[each]["text"]))))

In [16]:
df_wirecard.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,704044765566930948,42056054,2016-02-28 20:45:27+00:00,704044765566930948,en,0,0,0,0,Hootsuite,None,None,Cloud-based payment on HCE is a real gamechang...
1,704037775868358656,2992824844,2016-02-28 20:17:40+00:00,704037775868358656,de,0,0,0,0,seed.mytweetsys.app.t00,None,None,Guck mal! job: Senior CRM Manager (m/w) bei Wi...
2,704000413834600452,1000523414,2016-02-28 17:49:13+00:00,704000413834600452,en,1,0,0,0,Twitter for iPhone,retweeted,703954164540379136,RT Singapore is on the Way to a Cashless Soci...
3,703984392327266304,88195332,2016-02-28 16:45:33+00:00,703984392327266304,en,1,0,0,0,Twitter Web Client,retweeted,703944694225620993,RT Investor-Update 26.02.16: Genworth Financi...
4,703954164540379136,42056054,2016-02-28 14:45:26+00:00,703954164540379136,en,1,0,1,0,Hootsuite,None,None,Singapore is on the Way to a Cashless Society:...


<h3>Löschen von Retweets</h3>

In [17]:
counter = 0
retweet_list = []
for each in range(len(df_wirecard)):
    if "retweeted" in df_wirecard.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wirecard = df_wirecard.drop(retweet_list)
df_wirecard = df_wirecard.reset_index(drop=True)

95717


In [18]:
len(df_wirecard)

108471

<h3>Löschen von doppelten Einträgen</h3>

In [19]:
df_wirecard = df_wirecard.drop_duplicates(subset=["tweet_id"])
df_wirecard = df_wirecard.reset_index(drop=True)

In [20]:
len(df_wirecard)

108218

In [21]:
df_wirecard.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,704044765566930948,42056054,2016-02-28 20:45:27+00:00,704044765566930948,en,0,0,0,0,Hootsuite,None,None,Cloud-based payment on HCE is a real gamechang...
1,704037775868358656,2992824844,2016-02-28 20:17:40+00:00,704037775868358656,de,0,0,0,0,seed.mytweetsys.app.t00,None,None,Guck mal! job: Senior CRM Manager (m/w) bei Wi...
2,703954164540379136,42056054,2016-02-28 14:45:26+00:00,703954164540379136,en,1,0,1,0,Hootsuite,None,None,Singapore is on the Way to a Cashless Society:...
3,703944694225620993,2994748821,2016-02-28 14:07:48+00:00,703944694225620993,en,1,0,1,0,Twitter Web Client,None,None,"Investor-Update 26.02.16: Genworth Financial, ..."
4,703943082119553024,315887594,2016-02-28 14:01:24+00:00,703943082119553024,en,0,0,0,0,NetworkedBlogs,None,None,"Investor-Update 26.02.16: Genworth Financial, ..."


<h2>Abfrage nach Chashtag Wirecard</h2>

In [22]:
df_wrcdf = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/WRCDF_EN/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wrcdf = df_wrcdf.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [23]:
for each in range(len(df_wrcdf)):
    df_wrcdf.loc[df_wrcdf["text"] == 
                      df_wrcdf.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$]', '', 
                                                                                       df_wrcdf.loc[each]["text"]))))

In [24]:
len(df_wrcdf)

1260

<h3>Löschen von Retweets</h3>

In [25]:
counter = 0
retweet_list = []
for each in range(len(df_wrcdf)):
    if "retweeted" in df_wrcdf.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wrcdf = df_wrcdf.drop(retweet_list)
df_wrcdf = df_wrcdf.reset_index(drop=True)

250


<h3>Löschen von doppelten Tweets</h3>

In [26]:
df_wrcdf = df_wrcdf.drop_duplicates(subset=["tweet_id"])
df_wrcdf = df_wrcdf.reset_index(drop=True)

In [27]:
len(df_wrcdf)

1010

<h1>Zusammenführen der Datensätze</h1>

In [28]:
list_data_frames = [df_manager, df_wirecard, df_wrcdf]
df = pd.concat(list_data_frames, ignore_index=True)

In [29]:
print(len(df_manager))
print(len(df_wirecard))
print(len(df_wrcdf))
print(len(df_manager)+len(df_wirecard)+len(df_wrcdf))
print(len(df))

8275
108218
1010
117503
117503


<h3>Löschen der doppelten Posts</h3>
<p>Dadurch, dass die Abfrage durch mehrere Querys erfolgt ist, befinden sich einige Tweets doppelt in dem Datensatz. Diese werden hier anhand der <b>"tweet_id"</b> gelöscht. </p>

In [30]:
df = df.drop_duplicates(subset=["tweet_id"])
df = df.reset_index(drop=True)

<p>Der neue Datensatz wird auf plausibilität geprüft.</p>

In [31]:
print(len(df))

111199


In [32]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703142204341026818,349287204,2016-02-26 08:58:59+00:00,703142204341026818,en,0,0,1,0,Twitter Web Client,None,None,1/2 I see Wirecard is rebounding after CEO Mar...
1,695255175334924288,428593285,2016-02-04 14:38:45+00:00,695255175334924288,en,0,0,0,0,IFTTT,None,None,Thank you for following us Markus Braun!
2,695237701386244098,95129352,2016-02-04 13:29:19+00:00,695237701386244098,en,1,0,1,0,Twitter Web Client,None,None,"Disrupt the Banks! (Jeff Stewart, Markus Braun..."
3,718385114170966016,17616154,2016-04-08 10:28:52+00:00,718385114170966016,en,0,0,0,0,Twitter Web Client,None,None,WDI wirecard CEO Markus Braun on Q4 2015 Resul...
4,718384477215567873,546713,2016-04-08 10:26:20+00:00,718384477215567873,en,1,0,3,0,Twitter Web Client,None,None,Wirecard Analyst Call transcript is available ...


In [33]:
df.tail()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
111194,1279541597009518592,121158213,2020-07-04 22:24:24+00:00,1279541597009518592,en,1,0,4,0,recurpost.com,None,None,㋡ GOOD QUESTION!! ↳ Why are you paying to be ...
111195,1279525520535965696,36842372,2020-07-04 21:20:31+00:00,1279525520535965696,en,1,0,0,0,Twitter for iPhone,None,None,"Would you invest in AMZN, MSFT, WRCDF, HTZ or..."
111196,1278331510291038215,1212530948,2020-07-04 14:35:50+00:00,1279423679735255042,en,0,0,0,0,Twitter Web App,replied_to,1278335952532975616,Is its symbol WRCDF? Great run on SHOP.
111197,1278553462322671616,1235225464278573056,2020-07-02 04:57:54+00:00,1278553462322671616,en,0,0,2,0,Twitter Web App,None,None,pcg emerges from Chapter 11This will make a lo...
111198,1278303159455232001,121158213,2020-07-01 12:23:17+00:00,1278303159455232001,en,0,0,0,0,recurpost.com,None,None,㋡ GOOD QUESTION!! ↳ Why are you paying to be ...


<p><b>Fertig</b> mit der Vorverarbeitung der Daten in deutsch. Es ist nun ein Datensatz verfügbar mit allen Tweets für den deutschsprachigen Raum. Es sind keine doppelten Tweets enthalten, solange diese keine gleiche tweet_id enthalten. Auch Nutzer und tweets, die nichts zu dem eigentlichen Thema Wirecard beitragen sind nicht mehr im Datensatz enthalten. Vereinzelt können natürlich unnütze Tweets vorhanden sein, aber die Nutzer mit vielen Interaktionen sind überprüft und auch die Begriffe, die besonders oft vorkommen sind überprüft worden, ob diese etwas mit dem Thema zu tun haben.<br>Jetzt muss das gleich noch mit den Englischen tweets geschehen und dann müssen Datensätze zur Überprüfung ausgegeben werden.<br> Um den zufälligen Datensatz zu erstellen sollte ich noch einen zufallszahlengenerator nutzen, sodass ich möglichst zufällige Tweets zum testen erhalte. Hier scheint die Pandas Funktion <b>"sample"</b> gut geeignet zu sein. </p>

In [34]:
df.sample(n = 20)

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
86891,1273985628712157185,1550350116,2020-06-19 14:26:58+00:00,1273985628712157185,en,0,0,0,0,TheOfficialBoard,None,None,WireCard - James Freis has joined the company ...
23990,964197782876839936,42056054,2018-02-15 18:00:40+00:00,964197782876839936,en,2,0,3,0,Agorapulse app,None,None,"“We use data-driven, innovative technologies s..."
60937,1276706720220434432,15709952,2020-06-29 00:09:00+00:00,1277393595109179392,en,0,1,0,0,Conversocial,replied_to,1276722635179786241,"Hello,We understand your worry. Please know y..."
90399,1273599943714930690,847825666049527809,2020-06-18 12:54:23+00:00,1273599943714930690,en,0,1,1,0,Twitter Web App,None,None,"Dear Ladies and Gentlemen,Given today's announ..."
98488,1284805802532970497,788776883533914112,2020-07-19 11:02:28+00:00,1284805802532970497,en,0,0,0,0,Germany News App,None,None,German Finance Watchdog Defends Agency Amid Wi...
38776,1119004275500253187,223238349,2019-04-19 18:19:38+00:00,1119304563863908352,en,0,0,2,0,Twitter for iPhone,replied_to,1119304157360476160,The wont stop for the the people who want...
51426,1225006705324503040,1019946738,2020-02-05 10:42:12+00:00,1225006705324503040,en,0,0,0,0,LinkedIn,None,None,Session yesterday on digitizing end-to-end onb...
86236,1274059527374200832,3897627953,2020-06-19 19:20:36+00:00,1274059527374200832,en,0,0,1,0,Twitter Web App,None,None,I just watched two seasons of Bad Banks on Net...
41919,1140552074549190656,14146817,2019-06-17 09:29:39+00:00,1140552074549190656,en,0,0,0,0,Microsoft Power Platform,None,None,"Wirecard, the global innovation leader for dig..."
109600,1278372207488651271,18949452,2020-07-01 16:57:39+00:00,1278372207488651271,en,8,9,8,3,SocialFlow,None,None,Head of German financial watchdog defends agen...


In [35]:
df.loc[df["tweet_id"] == 964197782876839936]["text"].values[0]

'“We use data-driven, innovative technologies so our clients can remove barriers between themselves and their customers.” This is WirecardUSA.    '

<h2>Finalen Datensatz als CSV ausgeben.</h2>

In [36]:
df.to_csv('Dataset_Wirecard.csv', sep=';', index=False)

<h2>Erstellen der CSV zur Überprüfung der Sentiment Ergebnisse</h2>

In [37]:
df.sample(n=10)[["tweet_id", "text"]]

,tweet_id,text
39369,1117842623081922563,. VP of Strategic Partnerships and Marketing K...
101963,1281650512375558145,Dichiarazione di Eidoo sull’insolvenza di Wire...
20305,898496713626968064,Investmentidee: Wirecard (Heiko Geiger)
2021,1183946652874399744,Tell it to catalan people!
11695,746967334686842880,Foreign media: Alibaba ready to invest in the ...
23718,968544515517829120,mobile payment and loyalty app AllianzPrime - ...
57341,1257043963686649856,"15/ This is not a forensic analysis, bu..."
41100,1126781428673536000,Job Stellenangebot - Wirecard Bank AG: Applic...
109095,1278550207571976197,"Now that has filed for insolvency, one can’t ..."
81144,1275064630142894080,Financial Daily Dose 6.22.2020 Top Story: In ...


In [39]:
df_sentiment_validation = df.sample(n = 1500)[["tweet_id", "text"]]
batch = 14
counter = 0 
for batch in range(0,15):
    start = 100*batch
    end = 100*batch+99
    csv_name = 'en_validation_{count}.csv'.format(count=batch)
    df_sentiment_validation.iloc[start:end].to_csv(csv_name, sep=';', index=False)
    print("Start bei {start} und ende bei {ende}. Name der Datei: {csv}".format(start=start, ende=end, csv=csv_name))

Start bei 0 und ende bei 99. Name der Datei: en_validation_0.csv
Start bei 100 und ende bei 199. Name der Datei: en_validation_1.csv
Start bei 200 und ende bei 299. Name der Datei: en_validation_2.csv
Start bei 300 und ende bei 399. Name der Datei: en_validation_3.csv
Start bei 400 und ende bei 499. Name der Datei: en_validation_4.csv
Start bei 500 und ende bei 599. Name der Datei: en_validation_5.csv
Start bei 600 und ende bei 699. Name der Datei: en_validation_6.csv
Start bei 700 und ende bei 799. Name der Datei: en_validation_7.csv
Start bei 800 und ende bei 899. Name der Datei: en_validation_8.csv
Start bei 900 und ende bei 999. Name der Datei: en_validation_9.csv
Start bei 1000 und ende bei 1099. Name der Datei: en_validation_10.csv
Start bei 1100 und ende bei 1199. Name der Datei: en_validation_11.csv
Start bei 1200 und ende bei 1299. Name der Datei: en_validation_12.csv
Start bei 1300 und ende bei 1399. Name der Datei: en_validation_13.csv
Start bei 1400 und ende bei 1499. Name 

<h1>Check User</h1>
<p>Im Englischen Datensatz wurde dies nur zur Überprüfung genutzt. Es gabe keine Nutzer mehr, die keinen Beitrag zum Thema geleistet haben</p>

<p>Laden der Daten</p>

In [ ]:
df_user = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_EN/user.csv", sep=";")

<h3>Nutzer die keinen Mehrwert bieten werden gelöscht.</h3>

In [ ]:
list_not_needed_author_ids = [3365669884, 536335486]
for each in list_not_needed_author_ids:
    df_manager = df_manager.drop(df_manager.loc[df_manager["author_id"] == each].index)
df_manager = df_manager.reset_index(drop=True)

<h2>Überprüfung und Check der Twitter Nutzer im Datensatz</h2>
<p>Dies ist für den Fall relevant, dass die Nutzer überprüft werden müssen. <br>Im aktuellen Zustand des Datensatzes nicht mehr notwendig.</p>

<p>Prüfen nach allen Nutzern mit dem Namen Markus</p>

In [ ]:
df_user.loc[df_user.name.str.lower().str.contains("markus")].drop_duplicates()

<p>Prüfen auf auffäliige Einträge bezüglich Jan Marksalek</p>

In [ ]:
df_user.loc[df_user.name.str.lower().str.contains("marsalek")].drop_duplicates()

<p>Auffällige User_IDs:</p>
<ul>Markus Braun
    <li>125696248 - MarkusHBraun</li>
    <li>827452635057881088 - _MarkusBraun( - CWO von Wirecard)</li>
    <li>1273592890506096642 - MarkusB59026577</li>
    <li>780754694419296256 - markuscbraun1</li>
</ul>
<ul>Jan (Marsalek)
</ul>

<p>Anzeigen der Tweets der "auffälligen" Nutzer</p>

In [ ]:
#3365669884 - MarkusB1980
pd.set_option('display.max_colwidth', None)
df_manager.loc[df_manager["author_id"] == 125696248]["text"]

In [ ]:
#827452635057881088 - _MarkusBraun( - CWO von Wirecard)
pd.set_option('display.max_colwidth', None)
df_manager.loc[df_manager["author_id"] == 827452635057881088]["text"]

In [ ]:
#536335486 - Advobraun
pd.set_option('display.max_colwidth', None)
df_manager.loc[df_manager["author_id"] == 1273592890506096642]["text"]

In [ ]:
#780754694419296256 - markuscbraun1
pd.set_option('display.max_colwidth', None)
df_manager.loc[df_manager["author_id"] == 780754694419296256]["text"]

<p>Wie erwartet sind alle Nutzerr mit auffälligen Namen bereits durch das durchgehen aller Tweets entfernt. Es finden sich nur noch zu einem Tweets, die aber etwas zum Thema beitragen.<br>Zu Jan Marsalek gibt es keine Nutzer.</p>
<p>Dabei handelt es sich um die einzigen Nutzer, zu denen noch Tweets nach der Vorverarbeitung übrig sind</p>

<p>Das Dataframe mit der Anzahl der jeweiligen Autoren wird um die Informationen aus dem Datensatz der User ergänzt.<br>Dabei soll gerpüft werden, ob auffällige Nutzer mit eigenartigen Namen auftauchen, die es zu prüfen gilt. Dabei wird vor allem auf Nutzer geachtet, die einen klaren Bezug zu anderen Theman haben oder die ggf. auffälligkeiten zu dem Thema haben könnten.</p>

In [ ]:
df_manager["author_id"].value_counts().to_frame().reset_index().join(
    df_user.set_index("user_id"), on="index").drop_duplicates(subset=["index"]).rename(columns={"index": "author_id", "author_id": "count"}).head(11)

<p>Stichprobenartige Prüfen der einzelnen Top Nutzer mit mehr als 50 Tweets auf deren Inhalte.</p>

In [ ]:
pd.set_option('display.max_colwidth', None)
for each in df_manager.loc[df_manager["author_id"] == 1091697475012849664].index:
    print(df_manager.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager.loc[df_manager["author_id"] == 1929600979].index:
    print(df_manager.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager.loc[df_manager["author_id"] == 3029833842].index:
    print(df_manager.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager.loc[df_manager["author_id"] == 300045480].index:
    print(df_manager.iloc[each]["text"], "\n")

In [ ]:
for each in df_manager.loc[df_manager["author_id"] == 223238349].index:
    print(df_manager.iloc[each]["text"], "\n")

In [ ]:
df_manager.loc[df_manager["author_id"] == 300045480]

<h1>Weitere Überprüfungen der Daten</h1>
<p>Hier sind die Überprüfungen aufgeführt, die vorgenommen werden mussten um den Datensatz vorzubereiten. Das eigentliche Löschen der nicht benötigten Daten ist bereits weiter oben vorgenommen worden.</p>

<h2>Überprüfung der Wörter in den Tweets</h2>
<p>Ich prüfe wie oft die einzelnen Wörter in den jeweiligen Tweets vorkommen um ggf. nicht passende Wörter zu ermitteln.</p>
<p>Ladeb von deutschen Stop Wörtern um diese zu filter, da diese keinen wirklichen Mehrwert bieten. <br> Anschließend wird das Hanover Tagger Modell geladen um die Lemmatisierung durchzuführen um Wörter auf die Grundform zu reduzieren.</p>

In [ ]:
german_stop_words = stopwords.words('german')
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

<p>Lemmatisierung um zu den jeweiligen Wörtern die Grundform zu erhalten. Dies wird mit den ganzen Tweets gemacht umd die Wörter im Kontext betrachten zu können (?). </p>

In [ ]:
tweet_list = df_manager["text"].tolist()
lemma_list = []

for tweet in tweet_list:
    lemma = [lemma for (word, lemma, pos) in tagger.tag_sent(tweet.split())]
    lemma_list.append(' '.join(lemma))

<p>Erstellen einer Liste mit allen Wörtern aus jeden einzelnen Tweet.</p>

In [ ]:
text_list = []
for each in lemma_list:
    for word in re.sub('\W+', ' ', each).lower().split():
        if word not in german_stop_words:
            text_list.append(word)

<p>Die Liste wird nun zu einer Pandas Serie um hier leichter Informationen zu erhalten. Im Anschluss wird die Serie zu einem Dataframe, bei dem die Spalten "Word" und "Count" heißen.</p>

In [ ]:
text_series = pd.Series(text_list)
df_word_count = text_series.value_counts().to_frame().reset_index().rename(columns={"index": "word", 0: "count"})

<p>Anzeigen von allen Wörtern, die öfter als 100 Mal in dem Datensatz vorkommen.</p>

In [ ]:
pd.set_option('display.max_rows', None)
df_word_count.loc[df_word_count["count"] >= 100]

<p>Überprüfung ob bei auffälligen Wörtern oder Kürzeln die Texte keinen Kontext zu Wirecard haben.</p>
<p>Auffällig Begriffe</p>
<lu>
    <li>FPÖ</li>
    <li>amp</li>
    <li>wdi</li>
</lu>

In [ ]:
for each in df_manager.loc[df_manager.text.str.lower().str.contains("fpö")]["text"]:
    print(each, '\n')

In [ ]:
for each in df_manager.loc[df_manager.text.str.lower().str.contains("amp")]["text"]:
    print(each, '\n')

In [ ]:
for each in df_manager.loc[df_manager.text.str.lower().str.contains("wdi")]["text"]:
    print(each, '\n')

<p>Der Begriff der in der Stichprobe keinen Bezug zu Wirecard hatte war <b>"FPÖ"</b>. Dementsprechend ist der Begriff oben in der ursprünlgichen Verarbeitung mit aufgenommen.</p>

<h2>Test prüfen nach Tweets ohne die Manager</h2>
<p>In diesem Abschnitt werden die die Tweets überprüft, in denen die Manager nicht vorkommen. Diese müssen Manuell überprüft werden und anschlißend auch im Kontext zugehöriger Tweets geprüft. <br>Wenn dabei herauskommen sollte, dass die Tweets nicht mit dem Thema zu tun haben, dann werden diese gelöscht. Dies wird bereits oben erledigt.</p>

<p>Ich prüfe auf verbleibende Tweets, in denen weder Markus Braun oder Jan Marsalek genannt werden. Ich prüfe dies auf verschiedene Schreibweisen um die Tweets zu ermitteln, in denen diese Namen nur in Links erwähnt wurden.</p>

In [ ]:
df_markus = df_manager.loc[df_manager.text.str.lower().str.contains("markus braun|markusbraun")].reset_index(drop=True)

In [ ]:
df_jan = df_manager.loc[df_manager.text.str.lower().str.contains("jan marsalek|janmarsalek")].reset_index(drop=True)

In [ ]:
df_markus.head()

In [ ]:
df_jan.head()

<p>Erhalten der Tweet_IDs und diese werden in einer Liste gespeichert.</p>

In [ ]:
list_markus_jan = []
for each in range(len(df_markus)):
    list_markus_jan.append(df_markus.iloc[each]["tweet_id"])
for each in range(len(df_jan)):
    list_markus_jan.append(df_jan.iloc[each]["tweet_id"])

In [ ]:
list_index = []
for each in list_markus_jan:
    list_index.append(df_manager.loc[df_manager["tweet_id"] == each].index[0])
    
df_man_with_mark_jan = df_manager.drop(list_index)
    #print(df_manager.loc[df_manager["tweet_id"] == each].index[0])
df_man_with_mark_jan = df_man_with_mark_jan.reset_index(drop=True)

In [ ]:
df_with_wirecard = df_man_with_mark_jan.loc[df_man_with_mark_jan.text.str.lower().str.contains("wirecard")]

In [ ]:
list_with_wirecard = []
for each in range(len(df_with_wirecard)):
    list_with_wirecard.append(df_with_wirecard.iloc[each]["tweet_id"])

In [ ]:
list_index = []
for each in list_with_wirecard:
    list_index.append(df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == each].index[0])
    #print(df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == each].index[0])
df_man_with_mark_jan = df_man_with_mark_jan.drop(list_index)
df_man_with_mark_jan = df_man_with_mark_jan.reset_index(drop=True)

<p>Prüfen von jedem Tweet, ob dieser einen Bezug zum Thema hat</p>

In [ ]:
for each in range(len(df_man_with_mark_jan)):
    print(df_man_with_mark_jan.iloc[each]["tweet_id"], ": ", df_man_with_mark_jan.iloc[each]["text"])

In [ ]:
print(len(df_man_with_mark_jan.drop_duplicates(subset=["text"])))

<p>Jeden Tweet, der als "auffällig" erkannt wurde mit zugehörigen Tweets manuell überprüfen</p>

In [ ]:
#Text Check up for Tweets without context.
tweet = 1130213579490234370
print("Original: ", df_man_with_mark_jan.loc[df_man_with_mark_jan["tweet_id"] == tweet]["text"].values[0], "\n")
for each in df_manager.loc[(
    df_manager["tweet_id"] == tweet) | (
    df_manager["conversation_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0]) | (
    df_manager["referenced_tweet_id"] == tweet) | (
    (df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager["referenced_tweet_id"] != "None")) | (
    df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0])]["text"].values:
    print(each, "\n")

<p>Ermittlung aller Tweets, die nichts mit dem Thema zu tun haben und die nach der manuellen Prüfung nicht in den Datensatz gehören. Alle Tweet_Ids werden in einer Liste gespeichert und im Anschluss gelöscht. Mit den bereits bekannten Tweet_IDs, wird dieser Prozess bereits weiter oben ausgeführt im Abschnitt "Tweets Löschen ohne Kontext"</p>

In [ ]:
#Text Check up for Tweet_id for removal
list_tweet_id = [701784983829143553, 909831998537158657, 909733713302441984, 909730731932368897, 1115628103487954944, 
                1115596390133051393, 1112624842493358080, 1131274228521742336, 1130213579490234370, 1207617572126044160, 
                1265176607905333248, 1273949458808680449, 1285646389771481088, 1285547240447246338, 1285515450584096768, 
                1285497151125151745, 1284925162836709378, 1279724802782216192]
list_tweet_id_remove = []
for tweet in list_tweet_id:
    for each in df_manager.loc[(
        df_manager["tweet_id"] == tweet) | (
        df_manager["conversation_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0]) | (
        df_manager["referenced_tweet_id"] == tweet) | (
        (df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager["referenced_tweet_id"] != "None")) | (
        df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0])]["tweet_id"].values:
        print(each, "\n")
        if each not in list_tweet_id_remove:
            list_tweet_id_remove.append(each)

<h1>TEST</h1>
<p>Hier werden unterschieldiche Funktionen getestet.</p>

In [ ]:
df.sample(n = 100)

In [ ]:
test_str_strInStr = "Das ist der Test ob Superkicker korrekt erkannt wird."

In [ ]:
if "kicker" in test_str_strInStr:
    print("Superkicker")

In [ ]:
for each in range(0,15):
    print(each)

In [ ]:
df_manager.loc[df_manager.text.str.lower().str.contains(("@_markusbraun | markusbraun"))].index

In [ ]:
list_single_tweet_id = [719940834028863489, 771966867707682816, 919891703116566529, 1274189525875593218]
for each in list_single_tweet_id:
    if each in df_manager.loc[df_manager["tweet_id"] == each].values:
        print("ok")

In [ ]:
df_manager.loc[df_manager.text.str.lower().str.contains("artist")]

In [ ]:
if 719940834028863489 in df_manager.loc[df_manager["tweet_id"] == 719940834028863489].values:
    print("ok")

In [ ]:
list_tweet_id = [719940834028863489, 771966867707682816, 919891703116566529, 730315192559673344, 1085012780053336064, 748146758232879104,
                 1275383293504978945, 1121414883512393728, 1288961015351316480, 1284465825915838464, 955660777960230917, 962669158923153409,
                 1016245557021507584, 1074388350734340097, 1115926018051792898, 1132148944031457280, 1130223973701246976, 1174984360669278209,
                 1220356450809188353, 1255176912202108929]
print(len(list_tweet_id))
list_new_tweet_id = []
for tweet_id in list_tweet_id:
    for each in df_manager.loc[df_manager["text"] == df_manager.loc[df_manager["tweet_id"] == tweet_id]["text"].values[0]]["tweet_id"].values:
        if each not in list_new_tweet_id:
            list_new_tweet_id.append(each)

print(len(list_new_tweet_id))

In [ ]:
print(list_new_tweet_id)

In [ ]:
df_manager.loc[(df_manager["tweet_id"] == 719940834028863489 ) | (df_manager["tweet_id"] == 1166388152786984963)]

In [ ]:
df_manager = df_manager.append(df_manager.loc[df_manager["tweet_id"] == 719940834028863489], ignore_index=True)

In [ ]:
df_manager.loc[df_manager["tweet_id"] == 719940834028863489]

In [ ]:
df_manager.at[17844, "tweet_id"] = 1

In [ ]:
df_manager.iloc[17844]

In [ ]:
list_tweet_id = []
not_used_words_list = ["meiko", "calisthenics", "pulled pork", "boxing", " fom", "dlr", "eurocity", "m.o.s.s", "markus_braun_"]
for each in range(len(df_manager)):
    for word in not_used_words_list:
        if word in df_manager.iloc[each]["text"].lower():
            if df_manager.iloc[each]["tweet_id"] not in list_tweet_id:
                list_tweet_id.append(df_manager.iloc[each]["tweet_id"])


#list_tweet_id_remove = []
for tweet in list_tweet_id:
    for each in df_manager.loc[(
        df_manager["tweet_id"] == tweet) | (
        df_manager["conversation_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0]) | (
        df_manager["referenced_tweet_id"] == tweet) | (
        (df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager["referenced_tweet_id"] != "None")) | (
        df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0])]["tweet_id"].values:
        #print(each, "\n")
        if each not in list_tweet_id:
            list_tweet_id.append(each)

In [ ]:
for each in list_tweet_id:
    df_manager = df_manager.drop(df_manager.loc[df_manager["tweet_id"] == each].index[0])
df_manager = df_manager.reset_index(drop=True)

In [ ]:
len(df_manager)

In [ ]:
print(len(list_tweet_id))
print(len(tweet_id_without_context_list))

In [ ]:
list(set(list_tweet_id) - set(tweet_id_without_context_list))

In [ ]:
for each in list_tweet_id:
    print(each, ": ", df_manager.loc[df_manager["tweet_id"] == each]["text"].values[0])